This script is to add data general. 
Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform('local')

<IPython.core.display.Javascript object>

C:\Users\pratama\Documents\GitHub\MESSAGEix\message_ix\message_ix\reporting\__init__.py:98: FutureWarning: Importing from genno.computations will be deprecated in a future version; use genno.operator instead.
  ("tom:nl-t-yv-ya", (genno.computations.add, "fom:nl-t-yv-ya", "vom:nl-t-yv-ya")),


This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [2]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "add_tech",
    "introducing the add_tech feature on MESSAGEix",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"


In [30]:
# Read DACCS technology data
with open('tech_data.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)

# Set up dictionary of parameter indices list
par_idx = {}
data = {}

# Create dicitonary of parameter indices and data
for tech in set(tech_data):
    # add vintage and active years and update tech_data for each tech
    years_vtg_act = scen.vintage_and_active_years()
    years_vtg_act = years_vtg_act[years_vtg_act['year_vtg'] >= tech_data[tech]['year_init']]
    tech_data[tech]['year_vtg']= years_vtg_act['year_vtg'].to_list()
    tech_data[tech]['year_act']= years_vtg_act['year_act'].to_list()

    # collect parameter indices and update data
    par_idx.update({tech: {name: {idx: [] for idx in list(MESSAGE_ITEMS[tech_data[tech][name]['par_name']].get('idx_names'))} 
                           for name in set(tech_data[tech]) - set(['year_init','year_vtg','year_act'])
                          }})
    
    data.update({tech: {name: [] for name in list(par_idx[tech].keys())}})


In [31]:
# If those are not provided, then this block of code is needed to retrieve them from the data input
regions = []
emissions = []
times = []
modes = []
commodities = []
levels = []
relations = []

set_elements_dict = {
    'node_loc'   :{'data': regions,     'name':'node'},
    'emission'   :{'data': emissions,   'name':'emission'},
    'mode'       :{'data': modes,       'name':'mode'},
    'time'       :{'data': times,       'name':'time'},
    'commodity'  :{'data': commodities, 'name':'commodity'},
    'level'      :{'data': levels,      'name':'level'},
    'time_origin':{'data': times,       'name':'time'},
    'time_dest'  :{'data': times,       'name':'time'},
    'relation'   :{'data': relations,   'name':'relation'},
    'node_rel'   :{'data': regions,     'name':'node'},
}   

# investment cost


In [35]:
# Create DataFrame for all parameters

for tec, val in data.items():
    for name in val.keys():
        # tec = 'DACCS'
        # name = 'emission_factor_co2'

        kwargs = {}
        if all(idx in par_idx[tec][name] for idx in ['year_vtg','year_act']):
            kwargs = {
                'year_vtg': tech_data[tec]['year_vtg'],
                'year_act': tech_data[tec]['year_act']
            }
        elif 'year_vtg' in par_idx[tech][name]:
            kwargs = {
                'year_vtg': sorted(set(tech_data[tec]['year_vtg']))
            }
        else:
            kwargs = {
                'year_act': sorted(set(tech_data[tec]['year_act']))
            }
            # if 'year_rel' is present, the values are assumed from 'year_act' values
            if 'year_rel' in par_idx[tec][name]:
                kwargs.update(
                    {'year_rel': sorted(set(tech_data[tec]['year_act']))}
                )

        df = make_df(tech_data[tec][name]['par_name'],
                    technology=tec,
                    value=tech_data[tec][name]['value'],
                    unit=tech_data[tec][name]['unit'],
                    **kwargs)    

        # create empty dataframe
        idx_exp = [e for e in par_idx[tec][name] 
                if e not in ['technology','year_vtg','year_act',
                                'node_origin','node_dest',
                                'time_origin','time_dest',
                            ]]

        for idx in idx_exp:
            default = (
                {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))[1:]} 
                if idx in ['node_loc','mode']
                else {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))}
            )
            
            listidx = list(tech_data[tec][name].get(idx,default))
            listdfidx = []
            for e in listidx:
                df1 = df.copy()
                df1[idx] = [e] * len(df)
                listdfidx.append(df1)
            df = pd.concat(listdfidx, ignore_index=True)

        # assigning values for node and time related indices
        for idx in df.columns:
            if idx in ['node_origin','node_dest']:
                df[idx] = df['node_loc']
            if idx in ['time_origin','time_dest']:
                df[idx] = df['time']

        # Calculate values of row-by-row multipliers
        mult = []
        for i in range(len(df)):
            # node_loc factor
            _node_loc = (tech_data[tec]
                        .get(name,{})
                        .get('node_loc',{})
                        .get(df.get('node_loc',{}).get(2),1)
                        )

            # year_vtg factor
            # _year_vtg = (1+rate)**delta_years
            _year_vtg = (((1+tech_data[tec]
                        .get(name,{})
                        .get('year_vtg',{})
                        .get('rate',0))
                        **(df['year_vtg'][i] - tech_data[tech]['year_init']))
                        if 'year_vtg' in df.columns else 1
                        )

            # year_act factor
            # _year_act = (1+rate)**(year_act-year_vtg) if both years present
            # _year_act = (1+rate)**(year_act-first_active_year) if no year_vtg
            _year_act = (((1+tech_data[tec]
                            .get(name,{})
                            .get('year_act',{})
                            .get('rate',0))
                        **(df['year_act'][i]
                            -(df['year_vtg'][i] 
                            if 'year_vtg' in df.columns 
                            else tech_data[tech]['year_init'])))
                        if 'year_act' in df.columns else 1
                        )

            # get mode multiplier from model_data
            _mode = (tech_data[tec]
                    .get(name,{})
                    .get('mode',{})
                    .get(df.get('mode',{}).get(i),1)
                    )

            mult.append(
                    np.prod([_node_loc, _year_vtg, _year_act, _mode,])
                )

        # index adjusted df
        value = df['value'] * mult
        value = [round(e,3) for e in value]
        df['value'] = value

        data[tec][name] = df

In [36]:
for tec,val in data.items():
    with pd.ExcelWriter(f'{tec}.xlsx', engine='xlsxwriter', mode='w') as writer:
        for sheet_name, sheet_data in val.items():
            sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

# Below is old script draft

In [ ]:
# DONOT DELETE
tec = 'DACCS'
name = 'emission_factor_co2'

kwargs = {}
if all(idx in par_idx[tec][name] for idx in ['year_vtg','year_act']):
    kwargs = {
        'year_vtg': tech_data[tec]['year_vtg'],
        'year_act': tech_data[tec]['year_act']
    }
elif 'year_vtg' in par_idx[tech][name]:
    kwargs = {
        'year_vtg': sorted(set(tech_data[tec]['year_vtg']))
    }
else:
    kwargs = {
        'year_act': sorted(set(tech_data[tec]['year_act']))
    }
    # if 'year_rel' is present, the values are assumed from 'year_act' values
    if 'year_rel' in par_idx[tec][name]:
        kwargs.update(
            {'year_rel': sorted(set(tech_data[tec]['year_act']))}
        )

df = make_df(tech_data[tec][name]['par_name'],
             technology=tec,
             value=tech_data[tec][name]['value'],
             unit=tech_data[tec][name]['unit'],
             **kwargs)    

# create empty dataframe
idx_exp = [e for e in par_idx[tec][name] 
           if e not in ['technology','year_vtg','year_act',
                        'node_origin','node_dest',
                        'time_origin','time_dest',
                       ]]

for idx in idx_exp:
    default = (
        {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))[1:]} 
        if idx in ['node_loc','mode']
        else {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))}
    )
    
    listidx = list(tech_data[tec][name].get(idx,default))
    listdfidx = []
    for e in listidx:
        df1 = df.copy()
        df1[idx] = [e] * len(df)
        listdfidx.append(df1)
    df = pd.concat(listdfidx, ignore_index=True)

# assigning values for node and time related indices
for idx in df.columns:
    if idx in ['node_origin','node_dest']:
        df[idx] = df['node_loc']
    if idx in ['time_origin','time_dest']:
        df[idx] = df['time']

# Calculate value row-by-row multiplier
mult = []
for i in range(len(df)):
    # node_loc factor
    _node_loc = (tech_data[tec]
                  .get(name,{})
                  .get('node_loc',{})
                  .get(df.get('node_loc',{}).get(2),1)
                 )

    # year_vtg factor
    # _year_vtg = (1+rate)**delta_years
    _year_vtg = (((1+tech_data[tec]
                   .get(name,{})
                   .get('year_vtg',{})
                   .get('rate',0))
                  **(df['year_vtg'][i] - tech_data[tech]['year_init']))
                  if 'year_vtg' in df.columns else 1
                )

    # year_act factor
    # _year_act = (1+rate)**(year_act-year_vtg) if both years present
    # _year_act = (1+rate)**(year_act-first_active_year) if no year_vtg
    _year_act = (((1+tech_data[tec]
                    .get(name,{})
                    .get('year_act',{})
                    .get('rate',0))
                   **(df['year_act'][i]
                     -(df['year_vtg'][i] 
                       if 'year_vtg' in df.columns 
                       else tech_data[tech]['year_init'])))
                  if 'year_act' in df.columns else 1
                )

    # get mode multiplier from model_data
    _mode = (tech_data[tec]
             .get(name,{})
             .get('mode',{})
             .get(df.get('mode',{}).get(i),1)
            )

    mult.append(
            np.prod([_node_loc, _year_vtg, _year_act, _mode,])
        )

# index adjusted df
value = df['value'] * mult
value = [round(e,3) for e in value]
df['value'] = value

df

# Close the database

In [ ]:
mp.close_db()